# Gradient descent and Newton's method

(This exercise originates from S. Boyd's course Convex Optimization)

Consider the unconstrained problem
  
  $$\begin{array}{ll}minimize\,\,\, f(x) = - \sum_{i=1}^m \log(1-a_i^T x)- \sum_{i=1}^n \log(1 - x_i^2),\end{array}$$
  
  with variable $x \in \mathbf{R}^n$, and $\mathbf{dom} f = \{x \;|\; a_i^Tx \le 1, ~i=1, \ldots, m, ~|x_i| \le 1, ~i=1, \ldots, n\}$. This is the problem of computing the analytic center of the set of linear inequalities
  
  $$a_i^Tx \leq 1, \quad i=1, \ldots, m, \qquad |x_i|\leq 1,\quad i=1, \ldots, n.$$
 
 Note that we can choose $x^{(0)}=0$ as our initial point. You can generate instances of this problem by choosing $a_i$ from some distribution on $\mathbf{R}^n$.
 

1. Use the gradient descent method to solve the problem. Use backtracking line search (BLS) to choose the step size and a stopping criterion of the form $\| \nabla f(x)\|_2^2 \leq \delta$. You need to choose the parameters $\alpha,\beta$ of BLS as well as $\delta$ in a reasonable way. Plot the objective function and step length versus iteration number. (Once you have determined $p^\star$ to high accuracy, you can also plot $f-p^\star$ versus iteration.) Experiment with the backtracking parameters $\alpha$ and $\beta$ to see their effect on the total number of iterations required. 
2. Repeat using Newton&#39;s method, again with BLS and this time with a stopping criterion based on the square $\lambda^2$ of Newton decrement. Produce the plots as for GD. Can you see the sharp distinction between the areas of slow and fast convergence for the Newton&#39;s method?

Some notes and tips:
* You can find the description of BLS in the slides for the Newton's method lecture. The exact same formulation can be used for GD.
* The domain of the objective is not $\mathbf{R}^n$. So, your BLS needs make sure that the chosen step length results in a point inside the domain (on top of satisfying the standard BLS condition). 
* You are going to have to find formulas for $\nabla f(x)$ and $\nabla^2 f(x)$ by hand using chain rule. If you really want to, you can use pytorch (or other similar tools) for this task, but it is probably not worth it. The GD code and the Newton code has to be your own.
* In the Newton's algorithm do not invert the Hessian, use ``numpy.linalg.solve`` (or other linear system solver) instead.

In [15]:
import math
import numpy as np
import random as rand
import matplotlib.pyplot as plt

In [16]:
def log_1_minus_dot_prod(ai, x, n):
    dot_prod = 0
    for i in range(n):
        dot_prod += ai[i] * x[i]
    return math.log(1 - dot_prod)

def f(x, A, n, m):
    msum = 0
    for i in range(m):
        msum += log_1_minus_dot_prod(A[i], x, n)
    nsum = 0
    for i in range(n):
        nsum += math.log(1 - x[i] ** 2)
    return 0 - msum - nsum

def deriviative_f(x, A, n, m):
    ret = []
    for j in range(n):
        sum = 0
        for i in range(m):
            sum += A[i][j] / (1 - A[i][j] * x[j])
        sum += 2 * x[j] / (1 - x[j] ** 2)
        ret.append(sum)
    return ret

def hessian_f(x, A, n, m):
    # define the hessian matrix as a 2D array of 0
    hessian = np.zeros((n, n))
    for i in range(n):
        sum = 0
        for j in range(m):
            sum += 2 * A[j][i] ** 2 / (1 - A[j][i] * x[i]) ** 2
        sum += 2 / (1 - x[i] ** 2) ** 2
        hessian[i][i] = sum
    return hessian

def dot_product(x, y):
    dot_prod = 0
    for i in range(len(x)):
        dot_prod += x[i] * y[i]
    return dot_prod


In [17]:
def generate_A(n, m):
    A = []
    for i in range(m):
        ai = []
        for j in range(n):
            ai.append(rand.uniform(-1, 1))
        A.append(ai)
    return A

In [18]:
def in_domain(x, A, n, m) -> bool:
    for i in range(n):
        if x[i] >= 1 or x[i] <= -1:
            return False
    for i in range(m):
        if dot_product(A[i], x) >= 1:
            return False
    return True

In [19]:
def BLS(alpha, beta, x, A, n, m, direction):
    ni = 1
    while True:
        new_x = [x[i] + ni * direction[i] for i in range(n)]
        if in_domain(new_x, A, n, m) and f(new_x, A, n, m) < f(x, A, n, m) + alpha * ni * dot_product(deriviative_f(x, A, n, m),direction):
            return ni
        ni *= beta
        
        

In [20]:
def gradient_descent(x, A, n, m, alfa = 0.25, beta = 0.5):
    objective_function_vals = []
    step_lenghts = []
    iteration = 0
    while True:
        direction = deriviative_f(x, A, n, m)
        direction = [0 - i for i in direction]
        #direction = [i / abs(i) for i in direction]
        ni = BLS(alfa, beta, x, A, n, m, direction)
        direction = [i * ni for i in direction] 
        x = [x[i] + direction[i] for i in range(n)]
        objective_function_vals.append(f(x, A, n, m))
        step_lenghts.append(ni)
        iteration += 1
        print(abs(f(x, A, n, m) - f([x[i] - direction[i] for i in range(n)], A, n, m)))
        if abs(f(x, A, n, m) - f([x[i] - direction[i] for i in range(n)], A, n, m)) < 0.001:
            return x, objective_function_vals, step_lenghts, iteration

In [ ]:
def Newtons_Method(x, A, n, m, alfa = 0.25, beta = 0.5):
    objective_function_vals = []
    step_lengths = []
    iteration = 0
    while True:
        gradient = deriviative_f(x, A, n, m)
        hessian = hessian_f(x, A, n, m)
        direction = np.linalg.solve(hessian, gradient)
        ni = BLS(alfa, beta, x, A, n, m, direction)
        direction = [i * ni for i in direction]
        x = [x[i] + direction[i] for i in range(n)]
        objective_function_vals.append(f(x, A, n, m))
        step_lengths.append(ni)
        if np.linalg.norm(gradient) ** 2 <= 0.0001:
            return x, objective_function_vals, step_lengths, iteration
    
    

In [ ]:
x = [0, 0, 0, 0, 0]
best_ofv = []
best_step = []
best_x = []
min_iter_num = 1000
for alpha in [0.25, 0.5, 0.75]:
    for beta in [0.5, 0.75, 0.9]:
        x, objective_function_vals, step_lenghts, iter_num = Newtons_Method(x, A, 5, 5, alfa=alpha, beta=beta)
        if iter_num < min_iter_num:
            min_iter_num = iter_num
            best_ofv = objective_function_vals
            best_step = step_lenghts
            best_x = x
            best_alpha = alpha
            best_beta = beta
print("Best alpha: ", best_alpha)
print("Best beta: ", best_beta)
print("Best x: ", best_x)
# plot the objective function values against the iteration number
plt.plot(range(len(best_ofv)), best_ofv)
plt.xlabel("Iteration number")
plt.ylabel("Objective function value")
plt.show()

# plot the step lengths against the iteration number
plt.plot(range(len(best_step)), best_step)
plt.xlabel("Iteration number")
plt.ylabel("Step length")
plt.show()
